<a href="https://colab.research.google.com/github/ohjisu320/study_AIs/blob/main/docs/quests/MLs/SpineSurgeryList_GridSearshCV_resampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quest
- resampling에 따른 F1 score 변화 관찰

## 데이터

In [50]:
import pandas as pd
df_SSL = pd.read_csv('SpineSurgeryList.csv')
df_SSL[:2]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [51]:
# ['성별','신장','체중', '흡연여부', '연령', '혈액형', '직업', '재발여부' ] # 재발여부를 target으로
df_SSL_extract = df_SSL[['성별','신장','체중', '흡연여부', '연령', '혈액형', '직업', '재발여부'  ]]
df_SSL_extract[:2]

,성별,신장,체중,흡연여부,연령,혈액형,직업,재발여부
0,2,163,60.3,0,66,RH+A,자영업,0
1,1,171,71.7,0,47,RH+A,운동선수,0


## 전처리

In [52]:
df_SSL_extract.isnull().sum()

성별        0
신장        0
체중        0
흡연여부      0
연령        0
혈액형       0
직업      415
재발여부      0
dtype: int64

In [53]:
df_SSL_extract.dropna(inplace=True)
df_SSL_extract.isnull().sum()

<ipython-input-53-a12510f4a7b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SSL_extract.dropna(inplace=True)


성별      0
신장      0
체중      0
흡연여부    0
연령      0
혈액형     0
직업      0
재발여부    0
dtype: int64

In [54]:
target = df_SSL_extract['재발여부']
Xs = df_SSL_extract.drop(columns=['재발여부'])
features = pd.get_dummies(Xs) # 사용 지양 / 문자를 onehotencoding한 것 처럼 만듦. 새로운 데이터 인식 못함.
features

,성별,신장,체중,흡연여부,연령,혈액형_RH+A,혈액형_RH+AB,혈액형_RH+B,혈액형_RH+O,직업_건설업,...,직업_사무직,직업_사업가,직업_예술가,직업_운동선수,직업_운수업,직업_의료직,직업_자영업,직업_주부,직업_특수전문직,직업_학생
0,2,163,60.3,0,66,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,171,71.7,0,47,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1,178,77.1,0,39,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,174,74.2,0,40,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,1,183,80.7,0,42,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,2,157,64.0,0,59,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,157,59.0,0,42,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1891,1,167,70.0,0,61,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1892,1,177,77.0,0,29,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


## 특성공학 - Cross Validation

In [55]:
 # GridSearchCV = Cross Validation + Hyper Parameter Tuning(parameter 자동대입)
 # model, dataset 모두에 적용
 ## model: hyper parameter 교차적용 == 반복적용 가능
 ## dataset: 알맞게 섞어서 대입
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

hyper_parameters = {'max_depth':range(2, 10),
              'min_samples_leaf': range(2, 10),
              'criterion': ['gini','entropy'],
              'class_weight': [None, 'balanced'],
              'min_samples_split': range(2, 10)}
estimator_model =DecisionTreeClassifier()
# grid_model = GridSearchCV(estimator_model, hyper_parameters)

# score 방식 지정
from sklearn.metrics import f1_score, make_scorer
scoring = make_scorer(f1_score)
grid_model = GridSearchCV(estimator_model, hyper_parameters
                          , scoring=scoring
                          , cv=5) # dataset fold : shuffle과 비슷 fold:접기?
grid_model

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

## 모델

In [56]:
## 소요시간 알려줌
# from datetime import datetime

# start = datetime.now()  # start date and time

# grid_model.fit(features, target)

# end = datetime.now()  # end date and time - set to current date and time

# duration = end - start  # duration as a timedelta object

# print("Duration: {0}".format(duration))

In [57]:
grid_model.fit(features, target)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

In [58]:
best_model = grid_model.best_estimator_

In [59]:
grid_model.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 9,
 'min_samples_leaf': 8,
 'min_samples_split': 9}

In [60]:
grid_model.scorer_

make_scorer(f1_score)

In [61]:
grid_model.best_score_

0.2693194344053862

## 평가

In [62]:
from sklearn.metrics import classification_report
target_predict = best_model.predict(features)
target.shape, target_predict.shape


((1479,), (1479,))

In [63]:
print(classification_report(target, target_predict)) # 일반평균과 가중평균의 차이가 크면, 데이터의 쏠림이 있다는 뜻.

              precision    recall  f1-score   support

           0       0.99      0.62      0.76      1302
           1       0.25      0.93      0.40       177

    accuracy                           0.66      1479
   macro avg       0.62      0.78      0.58      1479
weighted avg       0.90      0.66      0.72      1479



### ReSamplings
- Over Sampling
- Under Sampling
- Combine Sampling

#### Over Sampling
- 정확도 가장 낮음
- 평균차이 0.01

In [64]:
from collections import Counter
Counter(target)

Counter({0: 1302, 1: 177})

In [65]:
from imblearn.over_sampling import SMOTE
overSampling = SMOTE(sampling_strategy=0.8)
feature_over_sample, target_over_sample = overSampling.fit_resample(features, target)
feature_over_sample.shape, target_over_sample.shape

((2343, 26), (2343,))

In [66]:
from collections import Counter
Counter(target_over_sample)

Counter({0: 1302, 1: 1041})

In [67]:
best_model.fit(feature_over_sample, target_over_sample)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=9, min_samples_leaf=8, min_samples_split=9)

In [68]:
from sklearn.metrics import classification_report
target_over_sample_predict = best_model.predict(feature_over_sample)
target_over_sample.shape, target_over_sample_predict.shape

((2343,), (2343,))

In [69]:
print(classification_report(target_over_sample, target_over_sample_predict))

              precision    recall  f1-score   support

           0       0.86      0.96      0.91      1302
           1       0.94      0.81      0.87      1041

    accuracy                           0.89      2343
   macro avg       0.90      0.88      0.89      2343
weighted avg       0.89      0.89      0.89      2343



#### Under Sampling
- 정확도 가장 높음
- 평균차이 0.01

In [70]:
from imblearn.under_sampling import NearMiss
underSampling = NearMiss(sampling_strategy=0.8)
feature_under_sample, target_under_sample = underSampling.fit_resample(features, target)
feature_under_sample.shape, target_under_sample.shape

((398, 26), (398,))

In [71]:
best_model.fit(feature_under_sample, target_under_sample)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=9, min_samples_leaf=8, min_samples_split=9)

In [72]:
from sklearn.metrics import classification_report
target_under_sample_predict = best_model.predict(feature_under_sample)
target_under_sample.shape, target_under_sample_predict.shape

((398,), (398,))

In [73]:
print(classification_report(target_under_sample, target_under_sample_predict))

              precision    recall  f1-score   support

           0       0.82      0.77      0.80       221
           1       0.74      0.79      0.76       177

    accuracy                           0.78       398
   macro avg       0.78      0.78      0.78       398
weighted avg       0.78      0.78      0.78       398



### Combine Sampling
- 정확도 두번째로 높음
- 평균차이 0.04

In [76]:
from imblearn.combine import SMOTEENN
combineSampling = SMOTEENN(sampling_strategy=0.5)
features_combine_sample, target_combine_sample = combineSampling.fit_resample(features, target)
features_combine_sample.shape, target_combine_sample.shape

((1161, 26), (1161,))

In [79]:
from sklearn.metrics import classification_report
target_combine_sample_predict = best_model.predict(features_combine_sample)
target_combine_sample.shape, target_combine_sample_predict.shape

((1161,), (1161,))

In [80]:
print(classification_report(target_under_sample, target_under_sample_predict))

              precision    recall  f1-score   support

           0       0.82      0.77      0.80       221
           1       0.74      0.79      0.76       177

    accuracy                           0.78       398
   macro avg       0.78      0.78      0.78       398
weighted avg       0.78      0.78      0.78       398



In [81]:
from imblearn.combine import SMOTEENN
combineSampling = SMOTEENN(sampling_strategy={0:1302, 1:400})
features_combine_sample, target_combine_sample = combineSampling.fit_resample(features, target)
features_combine_sample.shape, target_combine_sample.shape

((981, 26), (981,))

In [82]:
from sklearn.metrics import classification_report
target_combine_sample_predict = best_model.predict(features_combine_sample)
target_combine_sample.shape, target_combine_sample_predict.shape

((981,), (981,))

In [83]:
print(classification_report(target_under_sample, target_under_sample_predict))

              precision    recall  f1-score   support

           0       0.82      0.77      0.80       221
           1       0.74      0.79      0.76       177

    accuracy                           0.78       398
   macro avg       0.78      0.78      0.78       398
weighted avg       0.78      0.78      0.78       398

